In [1]:
import numpy as np
import cv2 as cv
import pickle
import nbimporter
import locality_sensitive_hashing as lsh
import min_hashing as mh
import shingling as sh
import prime
import os, os.path

Importing Jupyter notebook from locality_sensitive_hashing.ipynb
Importing Jupyter notebook from min_hashing.ipynb
Importing Jupyter notebook from prime.ipynb
Importing Jupyter notebook from shingling.ipynb


Using TensorFlow backend.


In [27]:
def find_sobel_outliers(folder, sobel_shingles):
    deleted_file_names = []
    path, dirs, files = next(os.walk(folder))  
    for i in range(sobel_shingles.shape[0]):
        if np.count_nonzero(sobel_shingles[i]) == 0:
            deleted_file_names.append(files[i])
    return np.array(deleted_file_names)

In [5]:
def cleanup(files, folder):
    for f in files:
        os.remove(os.path.join(folder,f))

In [6]:
############################################################################################################################

In [14]:
with open('sobel_shingles.pickle', 'rb') as handle:
        sobel_shingles = pickle.load(handle)

In [ ]:
############################################################################################################################

In [28]:
deleted = find_sobel_outliers("../dataset_all", sobel_shingles.T)

In [29]:
deleted

['clock (236).JPEG',
 'clock (575).JPEG',
 'orange (1481).JPEG',
 'orange (1884).JPEG',
 'orange (2002).JPEG',
 'orange (2155).JPEG',
 'orange (2157).JPEG',
 'orange (2242).JPEG',
 'orange (2292).JPEG',
 'orange (2698).JPEG',
 'orange (2716).JPEG',
 'palmtree (402).JPEG',
 'palmtree (409).JPEG',
 'palmtree (634).JPEG',
 'speaker (1452).JPEG',
 'speaker (453).JPEG',
 'spider (288).JPEG']